In [9]:
from pathlib import Path
import rasterio
import pandas as pd
import geopandas as gpd
import numpy as np
from rasterstats import zonal_stats

BASEDIR = Path('.').resolve().parent.joinpath('datain/HTI/threshold_selection')
csv = BASEDIR.joinpath('HTI_thresholds.csv')
shp = BASEDIR.parent.parent.joinpath('shps/HTI/HTI_adm4.shp')
raster = BASEDIR.joinpath('HTI_tmp_rad.tif')
stats = zonal_stats(str(shp), raster, stats=['sum', 'mean', 'std'], geojson_out=True)
stats_geojson = gpd.GeoDataFrame.from_features(stats)
stats_geojson = stats_geojson[[f'NAME4', 'sum', 'mean', 'std']]


In [11]:
df = pd.read_csv(csv)
df = df.merge(stats_geojson, on='NAME4')

In [17]:
stats_geojson.columns = ['NAM', 'su', 'me', 'st']

In [19]:
df = pd.read_csv(csv)
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,std0.65,sum0.7000000000000001,mean0.7000000000000001,std0.7000000000000001,sum0.75,mean0.75,std0.75,sum0.8,mean0.8,std0.8
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2,2,2,2,2,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,3,3,3,3,3,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,4,4,4,4,4,4,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df = pd.read_csv(csv)
sums = [x for x in df.columns.values if x.startswith('sum')]
sums.remove('sum')
for column in sums:
    df_corr = df[['sum', column]]
    

            sum    sum0.0
0  13723.481445  0.107313
1  17201.796875  0.000000
2  11262.849609  0.000000
3  26420.322266  0.000000
4  15118.458008  0.534133
            sum   sum0.05
0  13723.481445  0.107313
1  17201.796875       NaN
2  11262.849609       NaN
3  26420.322266       NaN
4  15118.458008  0.534133
            sum    sum0.1
0  13723.481445  0.107313
1  17201.796875       NaN
2  11262.849609       NaN
3  26420.322266       NaN
4  15118.458008  0.404047
            sum  sum0.15000000000000002
0  13723.481445                     NaN
1  17201.796875                     NaN
2  11262.849609                     NaN
3  26420.322266                     NaN
4  15118.458008                0.404047
            sum    sum0.2
0  13723.481445       NaN
1  17201.796875       NaN
2  11262.849609       NaN
3  26420.322266       NaN
4  15118.458008  0.206711
            sum  sum0.25
0  13723.481445      NaN
1  17201.796875      NaN
2  11262.849609      NaN
3  26420.322266      NaN
4  15118.45